In [45]:
from e3nn import o3
import torch

In [46]:
# Compare the types of tensor products
irrep_in = "1x2o"
o3_type = o3.Irreps(irrep_in)
print(f"Dimension of the input: {o3_type.dim}")

x = torch.tensor([1.0, 0.0, 0.0, 1.0, 2.0])
y = torch.tensor([0.0, 1.0, 0.0, 2.0, 1.0])

# Full Tensor Product
print("Full Tensor Product")
ftp = o3.FullTensorProduct(o3_type, o3_type)
out = ftp(x, y)
print(f"Dimension of the output: {out.shape}")
print(f"Output irreps: {ftp.irreps_out}")

print("Reduced Tensor Product")
rtp = o3.ReducedTensorProducts("ij=ji", i=irrep_in)
out = rtp(x, y)
print(f"Dimension of the output: {out.shape}")
print(f"Output irreps: {rtp.irreps_out}")

Dimension of the input: 5
Full Tensor Product
Dimension of the output: torch.Size([25])
Output irreps: 1x0e+1x1e+1x2e+1x3e+1x4e
Reduced Tensor Product
Dimension of the output: torch.Size([15])
Output irreps: 1x0e+1x2e+1x4e
